# Portfolio Optimization

In [26]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import yfinance as yf

### 1. Preliminaries: get data, calculate returns and covariance 

In [28]:
stocks = "SPY NVDA GOOG  AMZN LLY".split()
stocks

['SPY', 'NVDA', 'GOOG', 'AMZN', 'LLY']

In [30]:
data = yf.download(stocks, "2023-10-28")['Close']
data.head(3)

[*********************100%***********************]  5 of 5 completed


Ticker,AMZN,GOOG,LLY,NVDA,SPY
Date,,,,,
2023-10-30 00:00:00+00:00,132.710007,125.750000,565.710022,41.160999,415.589996
2023-10-31 00:00:00+00:00,133.089996,125.300003,553.929993,40.779999,418.200012
2023-11-01 00:00:00+00:00,137.000000,127.570000,554.460022,42.325001,422.660004


In [32]:
returns = np.log(data).diff().dropna()
returns.head(3)

Ticker,AMZN,GOOG,LLY,NVDA,SPY
Date,,,,,
2023-10-31 00:00:00+00:00,0.002859,-0.003585,-0.021043,-0.009299,0.006261
2023-11-01 00:00:00+00:00,0.028955,0.017954,0.000956,0.037186,0.010608
2023-11-02 00:00:00+00:00,0.007780,0.007886,0.045533,0.027521,0.018983


In [34]:
cov = returns.cov() * 252
cov.head(3)

Ticker,AMZN,GOOG,LLY,NVDA,SPY
Ticker,,,,,
AMZN,0.067259,0.034787,0.022246,0.060516,0.021252
GOOG,0.034787,0.066136,0.012179,0.044871,0.017006
LLY,0.022246,0.012179,0.084416,0.052257,0.012277


### 2. Set initial weights and calculate expected return

In [36]:
exp_rets = returns.mean() * 252
exp_rets

Ticker
AMZN    0.358185
GOOG    0.304951
LLY     0.467874
NVDA    1.234320
SPY     0.335998
dtype: float64

In [38]:
weights = np.array([.1,.1,.1,.3, .4])

### 3. Portfolio metrics based on pre-optimization parameters

In [42]:
exp_ret = np.dot(exp_rets, weights)
std = np.dot(weights.T, np.dot(weights, cov)) ** .5
sharpe = (exp_ret - .05) / std
print(f"Expected Ret:\t{exp_ret:>10.2%}")
print(f"Volatility:\t{std:>10.2%}")
print(f"Sharpe:\t\t{sharpe:>10.2f}")

Expected Ret:	    61.78%
Volatility:	    22.90%
Sharpe:		      2.48


### 4. Helper functions for optimization

In [51]:
def port_metrics(weights):
    rfr = .05
    stock_returns = returns.mean() * 252
    exp_ret = np.dot(weights, stock_returns)
    std = np.sqrt(np.dot(weights.T, np.dot(weights, cov)))
    sharpe = (exp_ret - rfr) / std
    return exp_ret, std, sharpe

def min_sharpe(weights):
    return -port_metrics(weights)[2]

def check_sum(weights):
    return sum(weights) - 1

In [53]:
print(port_metrics(weights))

(0.6177959660731156, 0.22904227290750384, 2.4790007489246912)


### 5. Set optimization constraints

In [57]:
bounds = [(0,1) for weight in weights]
constraints = {"type": "eq", "fun": check_sum}

### 6. Run optimization algorithm

In [60]:
portfolio = minimize(min_sharpe, weights, method="SLSQP", bounds = bounds, constraints = constraints)

### 7. Display optimal portfolio weights

In [62]:
portfolio.x.round(2)

array([0.  , 0.  , 0.11, 0.16, 0.73])

### 8. Revised, optimal portfolio metrics 

In [64]:
port_metrics(portfolio.x)

(0.4905480367723618, 0.1676846198185485, 2.627241766412917)